In [1]:
import pandas as pd
import numpy as np

In [2]:
#load dataset

import os.path

datasetdir = os.path.join('.', 'dataset')
filename = 'games.csv'
path = os.path.join(datasetdir, filename)

data = pd.read_csv(path, index_col='id')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103536 entries, 56115 to 123757
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   category             103536 non-null  int64  
 1   rating               24272 non-null   float64
 2   rating_count         24272 non-null   float64
 3   aggregated_rating    11678 non-null   float64
 4   first_release_date   103536 non-null  int64  
 5   follows              18052 non-null   float64
 6   game_modes           73251 non-null   object 
 7   genres               93346 non-null   object 
 8   involved_companies   66673 non-null   object 
 9   keywords             56122 non-null   object 
 10  summary              97520 non-null   object 
 11  storyline            12658 non-null   object 
 12  multiplayer_modes    6890 non-null    object 
 13  name                 103536 non-null  object 
 14  platforms            103536 non-null  object 
 15  player_perspe

In [3]:
rmed = data['rating'].median()
armed = data['aggregated_rating'].median()
for idx in data.index:
    if np.isnan(data.loc[idx, 'rating']) and np.isnan(data.loc[idx, 'aggregated_rating']):
        data.loc[idx, 'rating'] = rmed
        data.loc[idx, 'aggregated_rating'] = armed
    elif np.isnan(data.loc[idx, 'rating']):
        data.loc[idx, 'rating'] = data.loc[idx, 'aggregated_rating']
    elif np.isnan(data.loc[idx, 'aggregated_rating']):
        data.loc[idx, 'aggregated_rating'] = data.loc[idx, 'rating']
    else:
        data.loc[idx, 'rating'] = rmed
        data.loc[idx, 'aggregated_rating'] = armed
        

data['rating_count'].fillna(value=0, inplace=True)
data['follows'].fillna(value=0, inplace=True)
data['game_modes'].fillna(value='1', inplace=True)


data['year'] = pd.to_datetime(data['first_release_date'], unit='s').dt.year
data['involved_companies'].fillna(value='1', inplace=True)
data['genres'].fillna(value='', inplace=True)
data['involved_companies'].fillna(value='', inplace=True)
data['keywords'].fillna(value='', inplace=True)
data['summary'].fillna(value='', inplace=True)
data['storyline'].fillna(value='', inplace=True)
data['player_perspectives'].fillna(value='', inplace=True)
data['themes'].fillna(value='', inplace=True)

data['campaigncoop'].fillna(value=False, inplace=True)
data['dropin'].fillna(value=False, inplace=True)
data['lancoop'].fillna(value=False, inplace=True)
data['offlinecoop'].fillna(value=False, inplace=True)
data['onlinecoop'].fillna(value=False, inplace=True)
data['splitscreen'].fillna(value=False, inplace=True)

data.drop(['category', 'multiplayer_modes', 'similar_games', 'offlinecoopmax', 'rating_count', 
           'offlinemax',  'onlinecoopmax', 'onlinemax', 'platform', 'first_release_date'], axis=1, inplace=True)

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tony\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

def lemmatizer(text):
    wl = WordNetLemmatizer()
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    lemmatized_sentence = []
    tokenizer = RegexpTokenizer(r'\w+[-]?\w+')
    words = tokenizer.tokenize(text)
    word_pos_tags = nltk.pos_tag(words)

    for idx, tag in enumerate(word_pos_tags):
        lemmatized_sentence.append(wl.lemmatize(tag[0], get_wordnet_pos(tag[1])))
    return " ".join(lemmatized_sentence)

In [6]:
data['keywords'] = data['keywords'].apply(lambda x: x if x == '' else lemmatizer(x))
data['summary'] = data['summary'].apply(lambda x: x if x == '' else lemmatizer(x))
data['storyline'] = data['storyline'].apply(lambda x: x if x == '' else lemmatizer(x))

In [8]:
import os.path

datasetdir = os.path.join('.', 'dataset')
filename = 'games.clean.csv'
path = os.path.join(datasetdir, filename)
data.to_csv(path)

In [9]:
pcgames = data[data['platforms'].apply(str.split).apply(lambda x: [int(i) for i in x]).apply(lambda x: 6 in x)]
pcgames.loc[[pcgames[(pcgames['year'] == i)]['follows'].idxmax() for i in range(1990, 2023)]]

,rating,aggregated_rating,follows,game_modes,genres,involved_companies,keywords,summary,storyline,name,platforms,player_perspectives,themes,campaigncoop,dropin,lancoop,offlinecoop,onlinecoop,splitscreen,year
id,,,,,,,,,,,,,,,,,,,,
191,88.510472,88.510472,23.0,1,2 9 13 15 31,20461 20462 22857 27611,puzzle solve magic sword fantasy dragon world ...,Long after the passing of the Second Shadow wh...,Prologue The event of the game be precede by 3...,Loom,3 6 13 14 16 63 150,2,17 22,False,False,False,False,False,False,1990
3192,70.000000,70.000000,116.0,1,8 31,35597 173812 173813,anime fantasy 16-bit platformers 1990s robot d...,Sonic the Hedgehog be 2D side-scrolling platfo...,,Sonic the Hedgehog,5 6 9 12 29 39 52,4,1 17 18,False,False,False,False,False,False,1991
4438,70.000000,70.000000,156.0,1 2 3 4,8 31,16119 71906 72690 126174,anime retro side scroller mech fantasy 16-bit ...,Dr Eggman aka Dr Robotnik have return turn hel...,,Sonic the Hedgehog 2,5 6 9 12 29 37 41,4,1 18,True,True,False,True,True,False,1992
673,70.000000,70.000000,345.0,1 2 3,5,19609 19610 19611 19612 19613 19614 19615 1961...,first person shooter fps 5d maze futuristic co...,In the future human have leave Earth and settl...,The player take the role of marine unnamed to ...,DOOM,3 6 7 12 13 14 19 24 30 32 39 48 49 50 58 62 1...,1,1 18 19,True,False,False,True,True,True,1993
312,70.000000,70.000000,148.0,1 2 3,5,19620 19625 19626 19628 19630 43081 43082 4308...,post-apocalyptic chainsaw hell blood health fi...,Let the Obsession begin Again This time the en...,Immediately follow the event in Doom the playe...,DOOM II: Hell on Earth,6 12 13 14 44 130 149,1,1 18 19,False,False,False,False,False,False,1994
647,85.250977,85.250977,119.0,1 2,11 15,1519 1520 2083 2084 91420,modern warfare helicopter health construction ...,Experience the game that start it all Enter gr...,,Command & Conquer,4 6 7 13 14 32,3,18 39,False,False,False,False,False,False,1995
333,81.570883,81.570883,161.0,1 2 3,5,1009 1011 103611 103612 103615 103616 103617 1...,alien chainsaw arena blood medieval sword fant...,Quake be first-person shooter video game devel...,The player take the role of protagonist know a...,Quake,3 4 6 13 14 16 32 55 240,1,1 18 19 22 27,True,True,False,False,True,False,1996
286,83.352983,83.352983,338.0,1 2 3,5,1068 1070 1071 1072 145781 145782 150184 150185,alien robot blood mech health first person sho...,Quake II be first-person shooter video game de...,,Quake II,3 4 6 7 12 14 16 240,1,1 18 19,False,False,False,False,False,False,1997
231,70.000000,70.000000,793.0,1 2,5 9 31,18908 42057 96883,alien assassin blood gravity health first pers...,Dr Gordon Freeman doesn speak but he get hellu...,Dr Gordon Freeman arrive late for work at 47 b...,Half-Life,3 6 8 14,1,1 18 19 20 39,False,False,False,False,False,False,1998
